In [1]:
'''
Author: Ngo Van Uc
Date: 23/08/2024
Gmail: ngovanuc.1508@gmail.com
'''

'\nAuthor: Ngo Van Uc\nDate: 23/08/2024\nGmail: ngovanuc.1508@gmail.com\n'

# 00. Introduction

In [2]:
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from ast import literal_eval
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 01. Load dataset

In [5]:
train_path = '/content/drive/MyDrive/Colab Notebooks/data/Train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/data/Test.csv'
dev_path = '/content/drive/MyDrive/Colab Notebooks/data/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


# 02. Preprocessing

In [6]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9}
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [7]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [8]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]] * 10

train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [9]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 03. Build model

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [11]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [12]:
dataset.__len__()

7786

In [13]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')
        self.lstm = nn.LSTM(768, 256, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(256*2, n_classes * n_labels_per_class)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        lstm_outputs, _ = self.lstm(bert_outputs.last_hidden_state)
        lstm_outputs = lstm_outputs[:, -1, :]
        output = self.classifier(lstm_outputs)
        return output.view(-1, 10, 4)

model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

# 04. Training model

In [14]:
import torch.optim as optim
import numpy as np


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/20, Train loss: 0.35701917280161893
Epoch 2/20, Train loss: 0.2543433616109942
Epoch 3/20, Train loss: 0.21643054834136727
Epoch 4/20, Train loss: 0.1879604785747107
Epoch 5/20, Train loss: 0.16580554699138939
Epoch 6/20, Train loss: 0.14717986552499893
Epoch 7/20, Train loss: 0.13283996137138265
Epoch 8/20, Train loss: 0.11881087757111575
Epoch 9/20, Train loss: 0.10798266530036926
Epoch 10/20, Train loss: 0.09831595127862827
Epoch 11/20, Train loss: 0.08895349209588901
Epoch 12/20, Train loss: 0.08093492969778773
Epoch 13/20, Train loss: 0.07498413566461701
Epoch 14/20, Train loss: 0.06887476303800176
Epoch 15/20, Train loss: 0.06336844679220624
Epoch 16/20, Train loss: 0.0588150595102528
Epoch 17/20, Train loss: 0.05458639443180767
Epoch 18/20, Train loss: 0.04975261287170881
Epoch 19/20, Train loss: 0.04557999273957168
Epoch 20/20, Train loss: 0.04312661612020075


In [16]:
path_save_model = '/content/drive/MyDrive/Colab Notebooks'
torch.save(model.state_dict(), path_save_model+'/model_v3.pth')

In [17]:
# Load model
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/model_v3.pth'))
# model.eval()

In [18]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên bộ dữ liệu kiểm thử - test

In [19]:
test_data = pd.read_csv(test_path)
test_data.head()

,index,comment,n_star,date_time,label
0,0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",5,5/2/2020,{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",5,14/05/2019,{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,5,26/03/2020,{DESIGN#Neutral};{SER&ACC#Positive};
3,3,Màn hình hơi lác khi chơi game. Game nặng thì ...,4,4/6/2019,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",4,12/5/2020,{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [20]:
max_len = 256
idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

Mọi thứ ok.mình thấy pin trâu chứ có sao đâu.mua đc 2 tuần rồi chỉ có cảm biến vân tay ko nhạy như mong đợi thôi.
{FEATURES#Negative};{BATTERY#Positive};{GENERAL#Positive};
Aspect Sentiment Prediction:
SCREEN: None
CAMERA: None
FEATURES: Negative
BATTERY: Positive
PERFORMANCE: None
STORAGE: None
DESIGN: None
PRICE: None
GENERAL: Positive
SER&ACC: None


In [21]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


All done!


In [22]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [23]:
def full_label(true_label, prediction):
    '''
    true_label: dict
    prediction: dict
     '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [24]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = []
for idx in range(len(predictions_on_test)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [25]:
# data thỉnh thoảng bị sai -..-
# bằng chứng là SER&ACC trong câu này phải là Positive nhưng nhãn là None
# hoặc GENERAL nên là Positive thì nhãn lại là None

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'Positive', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'Positive'}
{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'None', 'SER&ACC': 'None'}
Thật tuyệt máy qua mượt túi thích  như thế giới Di Động có chỗ giá khác túi không hiểu đánh giá 5sao cho thế giới Di Động phương Phú Châu 5sao nhân viên OK
true:  SCREEN None
predicted:  SCREEN None
true:  CAMERA None
predicted:  CAMERA None
true:  FEATURES None
predicted:  FEATURES None
true:  BATTERY None
predicted:  BATTERY None
true:  PERFORMANCE None
predicted:  PERFORMANCE Positive
true:  STORAGE None
predicted:  STORAGE None
true:  DESIGN None
predicted:  DESIGN None
true:  PRICE None
predicted:  PRICE None
true:  GENERAL None
predicted:  GENERAL Positive
true:  SER&ACC None
predicted:  SER&A

# 07. Đánh giá hiệu suất tổng thể mô hình

In [26]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx][key] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and true_full_labels[idx][key] == value:
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9341334077588613
Precision = 0.8790697674418605
Recall = 0.8849498327759198
f1 = 0.882


# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy, Precision, Recall, F1-score trên toàn bộ khía cạnh

In [27]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

print(f'Accuracy = {accuracy_score(true_, predicted_)}')
print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

Accuracy = 0.8099328859060403
Precision = 0.8923395445134575
Recall = 0.8976495090746801
F1-score = 0.8949866508454464


### đánh giá hiệu suất khía cạnh trên class SCREEN

In [28]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SCREEN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9608812949640287
Precision = 0.8421052631578947
Recall = 0.8327137546468402
f1 = 0.8373831775700935


### đánh giá hiệu suất khía cạnh trên class FEATURES

In [29]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'FEATURES'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9204136690647482
Precision = 0.8617886178861789
Recall = 0.8945147679324894
f1 = 0.8778467908902691


### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [30]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PERFORMANCE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.89568345323741
Precision = 0.8903654485049833
Recall = 0.9146757679180887
f1 = 0.9023569023569024


### đánh giá hiệu suất khía cạnh trên class GENERAL

In [31]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'GENERAL'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.8570143884892086
Precision = 0.8714185883997205
Recall = 0.9029688631426502
f1 = 0.8869132290184922


### đánh giá hiệu suất khía cạnh trên class BATTERY

In [32]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'BATTERY'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9671762589928058
Precision = 0.951969260326609
Recall = 0.9773175542406312
f1 = 0.9644768856447689


### đánh giá hiệu suất khía cạnh trên class STORAGE

In [33]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'STORAGE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9883093525179856
Precision = 1.0
Recall = 0.037037037037037035
f1 = 0.07142857142857142


### đánh giá hiệu suất khía cạnh trên class DESIGN

In [34]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'DESIGN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9505395683453237
Precision = 0.9022346368715084
Recall = 0.8115577889447236
f1 = 0.8544973544973545


### đánh giá hiệu suất khía cạnh trên class PRICE

In [35]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PRICE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9532374100719424
Precision = 0.8907563025210085
Recall = 0.9314586994727593
f1 = 0.9106529209621993


### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [36]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SER&ACC'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9024280575539568
Precision = 0.8455882352941176
Recall = 0.7757166947723441
f1 = 0.809146877748461


### đánh giá hiệu suất khía cạnh trên class CAMERA

In [37]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'CAMERA'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9676258992805755
Precision = 0.9417808219178082
Recall = 0.935374149659864
f1 = 0.938566552901024


# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [38]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_full_labels)):
    for key, value in true_full_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)

print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.7872656947337102


### Precisio, Recall, F1-score tình cảm trên toàn bộ data kiểm thử

In [39]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

Classification report:
                      precision    recall  f1-score   support

    BATTERY_Negative     0.7832    0.9130    0.8432       368
     BATTERY_Neutral     0.4098    0.2717    0.3268        92
        BATTERY_None     0.9806    0.9587    0.9695      1210
    BATTERY_Positive     0.8966    0.8917    0.8941       554
     CAMERA_Negative     0.6340    0.8713    0.7340       171
      CAMERA_Neutral     0.3438    0.1549    0.2136        71
         CAMERA_None     0.9768    0.9792    0.9780      1636
     CAMERA_Positive     0.9401    0.8613    0.8989       346
     DESIGN_Negative     0.5876    0.5938    0.5907        96
      DESIGN_Neutral     0.0000    0.0000    0.0000        28
         DESIGN_None     0.9598    0.9808    0.9702      1826
     DESIGN_Positive     0.8927    0.8504    0.8710       274
   FEATURES_Negative     0.7468    0.8932    0.8135       459
    FEATURES_Neutral     0.5000    0.0577    0.1034        52
       FEATURES_None     0.9495    0.9326    0

In [40]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")


Overall Precision, Recall, F1-Score:
Micro Precision: 0.9030
Micro Recall: 0.9030
Micro F1-Score: 0.9030
Macro Precision: 0.6745
Macro Recall: 0.6403
Macro F1-Score: 0.6386
Weighted Precision: 0.9000
Weighted Recall: 0.9030
Weighted F1-Score: 0.8995


### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [41]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.69      0.75      0.72       116
     Neutral       0.00      0.00      0.00        17
        None       0.98      0.98      0.98      1955
    Positive       0.81      0.84      0.83       136

    accuracy                           0.95      2224
   macro avg       0.62      0.64      0.63      2224
weighted avg       0.94      0.95      0.95      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [42]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.75      0.89      0.81       459
     Neutral       0.50      0.06      0.10        52
        None       0.95      0.93      0.94      1513
    Positive       0.77      0.70      0.73       200

    accuracy                           0.88      2224
   macro avg       0.74      0.65      0.65      2224
weighted avg       0.88      0.88      0.88      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [43]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.67      0.86      0.76       454
     Neutral       0.30      0.23      0.26       116
        None       0.90      0.87      0.89      1052
    Positive       0.90      0.80      0.84       602

    accuracy                           0.82      2224
   macro avg       0.69      0.69      0.69      2224
weighted avg       0.82      0.82      0.82      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [44]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.68      0.85      0.76       294
     Neutral       0.66      0.40      0.50        83
        None       0.83      0.78      0.81       843
    Positive       0.89      0.90      0.90      1004

    accuracy                           0.83      2224
   macro avg       0.77      0.73      0.74      2224
weighted avg       0.83      0.83      0.83      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [45]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.63      0.87      0.73       171
     Neutral       0.34      0.15      0.21        71
        None       0.98      0.98      0.98      1636
    Positive       0.94      0.86      0.90       346

    accuracy                           0.93      2224
   macro avg       0.72      0.72      0.71      2224
weighted avg       0.92      0.93      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [46]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.78      0.91      0.84       368
     Neutral       0.41      0.27      0.33        92
        None       0.98      0.96      0.97      1210
    Positive       0.90      0.89      0.89       554

    accuracy                           0.91      2224
   macro avg       0.77      0.76      0.76      2224
weighted avg       0.90      0.91      0.90      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [47]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         6
     Neutral       0.00      0.00      0.00         3
        None       0.99      1.00      0.99      2197
    Positive       1.00      0.06      0.11        18

    accuracy                           0.99      2224
   macro avg       0.50      0.26      0.27      2224
weighted avg       0.98      0.99      0.98      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [48]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.59      0.59      0.59        96
     Neutral       0.00      0.00      0.00        28
        None       0.96      0.98      0.97      1826
    Positive       0.89      0.85      0.87       274

    accuracy                           0.94      2224
   macro avg       0.61      0.61      0.61      2224
weighted avg       0.92      0.94      0.93      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [49]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.48      0.61      0.54        79
     Neutral       0.77      0.78      0.78       328
        None       0.98      0.96      0.97      1655
    Positive       0.72      0.71      0.71       162

    accuracy                           0.90      2224
   macro avg       0.74      0.77      0.75      2224
weighted avg       0.91      0.90      0.91      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [50]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.56      0.51      0.54       167
     Neutral       0.00      0.00      0.00        27
        None       0.92      0.95      0.93      1631
    Positive       0.89      0.86      0.88       399

    accuracy                           0.89      2224
   macro avg       0.59      0.58      0.59      2224
weighted avg       0.88      0.89      0.88      2224

